# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Part II Data Modeling in Cassandra

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION =
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### 1st Query: 
Give the artist, song title and song's length in the music app history that was heard during
sessionId = 338, and itemInSession = 4

In [8]:
#dataframe check on the csv
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


In [9]:
#Later this will be used as a cross reference for the table I build
q1= df.query("sessionId == '338' and itemInSession == '4'")
q1

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
444,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


#### Song_info Table:
This table is modeled to get the artist's name, song title, and song length from the song_info table. The PRIMARY KEY consists of a partition key (session_id) and a clustering column (item_in_session)
    

In [10]:
#create the table

query = "CREATE TABLE IF NOT EXISTS song_info"
query= query + "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [12]:
## Verify data to make sure song_info table is filled properly

result = session.execute("""SELECT artist, song_title, song_length FROM song_info WHERE session_id = 338 AND item_in_session = 4;""")

for row in result: 
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2nd Query:
Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [13]:
#dataframe query to check against NoSQL table query
q2 = df.query("sessionId == '182' and userId == '10'")
q2

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4704,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
4705,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
4706,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
4707,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


#### Session_info Table:
This table is modeled to get the artist's name, song title, and users first and last name from the session_info table. The PRIMARY KEY consists of a composite partition key (user_id, session_id) and clustering column (item_in_session), since songs are expected to be sorted by itemInSession.

In [14]:
#create table for session_info
query = "CREATE TABLE IF NOT EXISTS session_info"
query = query + "(user_id int, session_id int, item_in_session int, artist text, song_title text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [15]:
#load data for the table from csv
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_info (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))                 

In [16]:
##Verifying that the table 'session_info' has been filled properly
query = "SELECT artist, song_title, first_name, last_name FROM session_info WHERE session_id=182 AND user_id=10;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.first_name, row.last_name)

Down To The Bone Sylvie Cruz Keep On Keepin' On
Three Drives Sylvie Cruz Greece 2000
Sebastien Tellier Sylvie Cruz Kilometer
Lonnie Gordon Sylvie Cruz Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


### 3rd Query: 
Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [17]:
#dataframe query to check against NoSQL table query
q3 = df.query("song == 'All Hands Against His Own'")
q3

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


#### User_info Table:
This table is modeled to get the users first and last name who listened to the song 'All Hands Against His Own'. The PRIMARY KEY consists of a partition key (song_title) and clustering column (user_id)

In [18]:
#create user_info table
query = "CREATE TABLE IF NOT EXISTS user_info"
query = query + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY(song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)  
                    

In [19]:
#read all lines from the csv file and extract specific columns values
#insert those values into the 'user_info' table
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ##Assigning here the INSERT statements into the `query` variable
        query = "INSERT INTO user_info (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s,%s,%s,%s)"
        ##Selecting here which element of each line (needed Columns) we will insert in the Table 'SESSION_USER_INFO'
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [20]:
##Verifying that the table 'USER_INFO' has been filled properly
query = "SELECT first_name, last_name FROM user_info WHERE song_title = 'All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name, row.last_name)


Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
##Dropping the 'song_info' before closing out the session
query = "DROP TABLE IF EXISTS song_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [22]:
##Dropping the 'session_info' before closing out the session
query = "DROP TABLE IF EXISTS session_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [23]:
##Dropping the 'user_info' before closing out the session
query = "DROP TABLE IF EXISTS user_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection

In [24]:
session.shutdown()
cluster.shutdown()